In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/language-detection/Language Detection.csv


In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.simplefilter("ignore")

In [3]:
data = pd.read_csv("/kaggle/input/language-detection/Language Detection.csv")

In [4]:
data.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [5]:
x = data["Text"]
y = data["Language"]

LabelEncoder assigns a number for each language class

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
le.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [8]:
data_list = []
for text in x:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

fit nad transform x_train

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(x_train)

x_train = cv.transform(x_train).toarray()
x_test = cv.transform(x_test).toarray()

using naive bayes classifier

In [11]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [12]:
y_pred = model.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ac = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [14]:
print("Accuray is: ", ac)

Accuray is:  0.9772727272727273


In [15]:
# from sklearn.pipeline import Pipeline

# pipe = Pipeline([('vectorizer', cv), ('multinomialNB', model)])
# pipe.fit(x_train.astype(str), y_train)

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Re-do the split on raw text instead of vectorized x_train
x_train_text, x_test_text, y_train, y_test = train_test_split(data_list, y, test_size=0.2, random_state=42)

pipe = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('multinomialNB', MultinomialNB())
])

pipe.fit(x_train_text, y_train)


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [16]:
# Predictions
y_pred2 = pipe.predict(x_test_text)
from sklearn.metrics import accuracy_score
ac2 = accuracy_score(y_test, y_pred2)
print("Accuracy is: ", ac2)

Accuracy is:  0.9821083172147002


In [17]:
# y_pred2 = pipe.predict(x_test)
# ac2 = accuaracy_score(y_test, y_pred2)
# print('Accuracy is: ', ac2)

In [18]:
with open('trained_pipeline-0.1.0.pkl','wb') as f:
    pickle.dump(pipe, f)

In [19]:
text = "Hello, how are you?"
text = "Ciao, come stai?"
#text = "Hi, how are you?"
y = pipe.predict([text])
le.classes_[y[0]], y

('Italian', array([8]))